In [1]:
import sys
sys.path.append('../')

In [2]:
from models.indexer_model import IndexerModule
import lovely_tensors as lt
lt.monkey_patch()

In [3]:
im = IndexerModule(
    lattice_size=50000, 
    num_iterations=5,
    error_precision=0.0012,
    filter_precision=0.00075,
    filter_min_num_spots=6
)

In [4]:
import torch

initial_cell = torch.tensor([
    [78.3, 0., 0.],
    [0., 78.3, 0.],
    [0., 0., 37.96]
])

reciprocal_spots = torch.tensor([
    [-0.2740,  0.0319,  0.0440],
    [ 0.0142, -0.1314,  0.0099],
    [-0.0742, -0.2719,  0.0460],
    [ 0.0839, -0.0099,  0.0040],
    [-0.1303, -0.2077,  0.0346],
    [ 0.1650, -0.0412,  0.0165],
    [-0.0526, -0.0877,  0.0059],
    [ 0.0984,  0.1081,  0.0121],
    [-0.2149,  0.0320,  0.0270],
    [ 0.0421, -0.0937,  0.0060],
    [-0.0844,  0.1088,  0.0108],
    [-0.1012,  0.0388,  0.0066],
    [ 0.2421,  0.1052,  0.0402],
    [ 0.0258,  0.0735,  0.0034],
    [ 0.0420,  0.2995,  0.0531],
    [-0.0561, -0.0402,  0.0027],
    [-0.2467, -0.1653,  0.0512],
    [-0.1013,  0.0284,  0.0063],
    [-0.1430, -0.0305,  0.0121],
    [ 0.0715, -0.0967,  0.0082],
    [ 0.2114, -0.1623,  0.0410],
    [-0.0933,  0.1838,  0.0243],
    [ 0.0372,  0.2275,  0.0305],
    [ 0.1390, -0.1969,  0.0334],
    [-0.1088,  0.2650,  0.0475],
    [-0.0223,  0.2774,  0.0448],
    [-0.0937, -0.2198,  0.0328],
    [ 0.1032,  0.2682,  0.0478],
    [ 0.0218, -0.1213,  0.0086],
    [-0.2295, -0.2075,  0.0557],
    [ 0.0903, -0.3135,  0.0622],
    [ 0.0341,  0.0573,  0.0025],
    [-0.1653, -0.1234,  0.0243],
    [-0.1992,  0.1412,  0.0343],
    [-0.1745,  0.0341,  0.0180],
    [-0.0701,  0.0340,  0.0034],
    [-0.0242, -0.0830,  0.0042]
]).unsqueeze(0)


In [7]:
%%time
solution_successes, solution_bases, solution_indices, solution_errors, _ = im(
        reciprocal_spots,
        initial_cell,
        min_num_spots=8,
        angle_resolution=150,
        num_top_solutions=100
    )


for success, basis, solution_mask, claimed_error in zip(
    solution_successes[0], solution_bases[0], solution_indices[0], solution_errors[0]
):
    print("Solution basis:")
    print(basis)
    print("Magnitud of solution basis(a, b, c):")
    print(basis.norm(dim=-1))
    miller_indices = torch.round(reciprocal_spots @ basis.T).int()
    print("Miller indices:")
    print(miller_indices[0][solution_mask].p)
    
    break


Solution basis:
tensor[3, 3] n=9 x∈[-47.868, 60.921] μ=13.199 σ=38.865 [[-10.215, 60.921, -47.868], [-38.397, 38.311, 56.152], [32.850, 15.075, 11.966]]
Magnitud of solution basis(a, b, c):
tensor[3] x∈[38.073, 78.148] μ=64.764 σ=23.115 [78.148, 78.071, 38.073]
Miller indices:
tensor([[-18,  -5,  -6],
        [-13,  -1,  -7],
        [ -5,  -7,   5],
        [  5,   1,   5],
        [  7,   8,  -1],
        [  2,  -3,  10],
        [  4,   2,   2],
        [-10,   6, -10],
        [ -1,   5,  -5],
        [ -7,  -6,   1],
        [-14, -12,   5],
        [ 11,  12,   0],
        [ 12,   9,   5],
        [-15, -11,   2],
        [ 15,  17,   1],
        [ 15,  14,   4],
        [-14,  -3,  -6],
        [ 13,   9,   8],
        [ -8,  -5,  -1],
        [-13,   4, -10],
        [-23, -12,  -1],
        [  3,   1,   2],
        [ -7,   3,  -7],
        [  9,  15,  -4],
        [  3,   9,  -5],
        [ -5,  -2,  -2]], dtype=torch.int32)
CPU times: user 201 ms, sys: 57.1 ms, total: 258 ms


In [24]:
#saving the model for C++
sm = torch.jit.script(im)
sm.save("traced_index_model.pt")